In [2]:
import os
import glob
import pandas as pd
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics"

all_files = glob.glob(os.path.join(folder_path, "*.csv"))

df_list = []

for file_path in all_files:

    df_temp = pd.read_csv(file_path)

    file_name = os.path.splitext(os.path.basename(file_path))[0]

    df_temp["source_file"] = file_name

    df_list.append(df_temp)

df_combined = pd.concat(df_list, ignore_index=True)

output_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data.csv"

df_combined.to_csv(output_path, index=False)
print(f"Combined data saved to: {output_path}")


NameError: name 'drive' is not defined

In [ ]:
print(folder_path)


/content/drive/MyDrive/Reddit_Data_Collection_statistics


In [ ]:
import pandas as pd

# Provide the full path to all_data.csv in Google Drive
csv_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data.csv"
df = pd.read_csv(csv_path)

# Filter rows by text length in the 4th column
df_filtered = df[(df.iloc[:, 4] > 10) & (df.iloc[:, 4] <= 250)]

# Save output
output_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_filtered.csv"
df_filtered.to_csv(output_path, index=False)
print(f"Filtered data saved to: {output_path}")


Filtered data saved to: /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_filtered.csv


In [ ]:
import pandas as pd
import math

# 1. Read the filtered data
input_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_filtered.csv"
df = pd.read_csv(input_path)

# 2. Shuffle the rows
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 3. Split into multiple CSV files, each with up to 200 rows
max_rows = 200
total_rows = len(df_shuffled)

# Calculate how many chunks are needed
num_chunks = math.ceil(total_rows / max_rows)

for i in range(num_chunks):
    # Start and end index for the chunk
    start_index = i * max_rows
    end_index = min(start_index + max_rows, total_rows)

    # Slice the DataFrame
    chunk = df_shuffled.iloc[start_index:end_index]

    # Name the output file (e.g., part_1, part_2, ...)
    output_path = f"/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_{i+1}.csv"
    chunk.to_csv(output_path, index=False)
    print(f"Saved {output_path} with {len(chunk)} rows.")


Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_1.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_2.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_3.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_4.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_5.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_6.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_7.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_8.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_9.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_10.csv with 200 rows.
Saved /content/driv

In [ ]:
import pandas as pd
import requests

input_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_1.csv"
df = pd.read_csv(input_path)

# Define your prompts
prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter(): I pay attention to one or a few details and fail to see the whole picture.
Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
Overgeneralization: I take isolated negative cases and generalize them, transforming them in a never-ending pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What if?: I keep asking myself questions such as “what if something happens?”
Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
Jumping to conclusions (also called arbitrary inference): I draw conclusions (negative or positive) from little or no confirmatory evidence.
Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions: 1. what is the situation? Find out the facts that are objective; what is this person thinking or imagining? Find out the thoughts or opinions that are subjective. 2. what makes this person think the thought is true or is not true? Find out the reasoning processes that support and do not support these thoughts. 3. why does this person come up with such reasoning process supporting the thought? What’s the underlying cognition mode of it?
"""

prompt3 = """
Please first answer: if there is cognitive distortion in the speech; Answer ’yes’ or ’no’; Please then answer: Recognizing the specific types of the cognitive distortion in the speech. There may be one type of cognitive distortion or multiple types involved. If there are multiple types, please give the top 2 dominant ones. Please only give the distortion type names separated by comma.
"""

# Function to send text to DeepSeek and get responses
def analyze_text_with_deepseek(text):
    # Replace with your DeepSeek API endpoint and API key
    api_url = "https://api.deepseek.com/v1/analyze"
    headers = {
        "Authorization": "",
        "Content-Type": "application/json"
    }

    # Combine prompts with the text
    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"

    # Send request to DeepSeek
    response = requests.post(api_url, headers=headers, json={"prompt": full_prompt})

    if response.status_code == 200:
        return response.json()  # Assuming DeepSeek returns a JSON response
    else:
        return {"error": f"API request failed with status code {response.status_code}"}

# Apply the function to each row in the CSV
df['Analysis'] = df.iloc[:, 3].apply(analyze_text_with_deepseek)

# Save the results to a new CSV file
df.to_csv('analyzed_data.csv', index=False)

print("Analysis complete. Results saved to 'analyzed_data.csv'.")

Analysis complete. Results saved to 'analyzed_data.csv'.


In [ ]:
import pandas as pd
from openai import OpenAI

# Initialize the OpenAI client with DeepSeek's API
client = OpenAI(api_key="", base_url="https://openrouter.ai/api/v1")

input_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_2.csv"
df = pd.read_csv(input_path)

# Define your prompts
prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter(): I pay attention to one or a few details and fail to see the whole picture.
Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
Overgeneralization: I take isolated negative cases and generalize them, transforming them in a never-ending pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What if?: I keep asking myself questions such as “what if something happens?”
Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
Jumping to conclusions (also called arbitrary inference): I draw conclusions (negative or positive) from little or no confirmatory evidence.
Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions: 1. what is the situation? Find out the facts that are objective; what is this person thinking or imagining? Find out the thoughts or opinions that are subjective. 2. what makes this person think the thought is true or is not true? Find out the reasoning processes that support and do not support these thoughts. 3. why does this person come up with such reasoning process supporting the thought? What’s the underlying cognition mode of it?
"""

prompt3 = """
Please first answer: if there is cognitive distortion in the speech; Answer ’yes’ or ’no’; Please then answer: Recognizing the specific types of the cognitive distortion in the speech. There may be one type of cognitive distortion or multiple types involved. If there are multiple types, please give the top 2 dominant ones. Please only give the distortion type names separated by comma.
"""

# Function to send text to DeepSeek and get responses
def analyze_text_with_deepseek(text):
    # Combine prompts with the text
    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"

    # Send request to DeepSeek
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": full_prompt},
        ],
        stream=False
    )

    # Extract the response content
    return response.choices[0].message.content

# Apply the function to the fourth column (index 3) in each row
df['Analysis'] = df.iloc[:, 3].apply(analyze_text_with_deepseek)

# Save the results to a new CSV file
df.to_csv('analyzed_data.csv', index=False)

print("Analysis complete. Results saved to 'analyzed_data.csv'.")

Analysis complete. Results saved to 'analyzed_data.csv'.


In [9]:
import ollama
import pandas as pd

# Load the dataset
input_path = "~/Downloads/all_data_part_3.csv"  # Adjust path as needed
df = pd.read_csv(input_path)

# Define your prompts
prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter(): I pay attention to one or a few details and fail to see the whole picture.
Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
Overgeneralization: I take isolated negative cases and generalize them, transforming them in a never-ending pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What if?: I keep asking myself questions such as “what if something happens?”
Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
Jumping to conclusions (also called arbitrary inference): I draw conclusions (negative or positive) from little or no confirmatory evidence.
Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions: 1. what is the situation? Find out the facts that are objective; what is this person thinking or imagining? Find out the thoughts or opinions that are subjective. 2. what makes this person think the thought is true or is not true? Find out the reasoning processes that support and do not support these thoughts. 3. why does this person come up with such reasoning process supporting the thought? What’s the underlying cognition mode of it?
"""

prompt3 = """
Please first answer: if there is cognitive distortion in the speech; Answer ’yes’ or ’no’; Please then answer: Recognizing the specific types of the cognitive distortion in the speech. There may be one type of cognitive distortion or multiple types involved. If there are multiple types, please give the top 2 dominant ones. Please only give the distortion type names separated by comma.
"""

def analyze_text_with_ollama(text):
    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"
    
    response = ollama.chat(model="deepseek-r1:14b", messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": full_prompt}
    ])
    
    return response['message']['content']

# Apply function to dataset with progress tracking
total_rows = len(df)
for i, row in enumerate(df.iterrows(), start=1):
    text = row[1].iloc[3]  # Assuming text is in the fourth column
    df.at[row[0], 'Analysis'] = analyze_text_with_ollama(text)
    
    # Print progress
    print(f"Processed {i}/{total_rows} rows ({(i/total_rows)*100:.2f}%)", flush=True)

# Save results
df.to_csv('../data/analyzed_data.csv', index=False)
print("Analysis complete. Results saved to 'analyzed_data.csv'.")


Processed 1/200 rows (0.50%)
Processed 2/200 rows (1.00%)
Processed 3/200 rows (1.50%)
Processed 4/200 rows (2.00%)
Processed 5/200 rows (2.50%)
Processed 6/200 rows (3.00%)
Processed 7/200 rows (3.50%)
Processed 8/200 rows (4.00%)
Processed 9/200 rows (4.50%)
Processed 10/200 rows (5.00%)
Processed 11/200 rows (5.50%)
Processed 12/200 rows (6.00%)
Processed 13/200 rows (6.50%)
Processed 14/200 rows (7.00%)
Processed 15/200 rows (7.50%)
Processed 16/200 rows (8.00%)
Processed 17/200 rows (8.50%)
Processed 18/200 rows (9.00%)
Processed 19/200 rows (9.50%)
Processed 20/200 rows (10.00%)
Processed 21/200 rows (10.50%)
Processed 22/200 rows (11.00%)
Processed 23/200 rows (11.50%)
Processed 24/200 rows (12.00%)
Processed 25/200 rows (12.50%)
Processed 26/200 rows (13.00%)
Processed 27/200 rows (13.50%)
Processed 28/200 rows (14.00%)
Processed 29/200 rows (14.50%)
Processed 30/200 rows (15.00%)
Processed 31/200 rows (15.50%)
Processed 32/200 rows (16.00%)
Processed 33/200 rows (16.50%)
Proce